In [1]:
## installing packages
!python3 -m pip install -q "mxnet<2.0.0"
!python3 -m pip install -q autogluon
!python3 -m pip install -q graphviz --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.6.1 requires tokenizers<0.11,>=0.10.1, but you have tokenizers 0.9.4 which is incompatible.
kornia 0.5.5 requires numpy<=1.19, but you have numpy 1.19.5 which is incompatible.
gym 0.18.3 requires Pillow<=8.2.0, but you have pillow 8.3.2 which is incompatible.
allennlp 2.5.0 requires torch<1.9.0,>=1.6.0, but you have torch 1.9.0 which is incompatible.
allennlp 2.5.0 requires torchvision<0.10.0,>=0.8.1, but you have torchvision 0.10.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mxnet 1.8.0.post0 requires graphviz<0.9.0,>=0.8.1, but you have graphviz 0.17 which is incompatible.


In [2]:
# imports
import gc
import os
import shutil
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
## define configuration
PATH_TRAIN = '../input/tabular-playground-series-sep-2021/train.csv'
PATH_TEST = '../input/tabular-playground-series-sep-2021/test.csv'

time_limit = 20000 # (in secs. for train run: set as 300 which gave public score ~10)

In [4]:
train = pd.read_csv(PATH_TRAIN, index_col="id")
test = pd.read_csv(PATH_TEST, index_col="id")

In [5]:
target_loss = train.claim
train.drop(['claim'], axis=1, inplace=True)

In [6]:
train.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
id,,,,,,,,,,,,,,,,,,,,,
0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,86.489,...,0.11093,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770
1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,...,0.97673,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359
2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,15827.000,...,0.20102,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690
3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,-36.837,...,-0.01182,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486
4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,144.120,...,0.92739,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049


In [7]:
## imports
from autogluon.tabular import TabularPredictor, TabularDataset

In [8]:
## run model
train['target'] = target_loss

model_ag = TabularPredictor(label='target')

model_ag.fit(train_data=train, 
              time_limit=time_limit, 
              presets='best_quality',
              num_stack_levels = 3,
              num_bag_folds = 5,
              num_bag_sets = 1,)

del train['target']

In [9]:
## check leaderboard
model_ag.leaderboard()

                      model  score_val  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   0.772486     412.501170   9597.861271                3.178522         282.661976            3       True         12
1           LightGBM_BAG_L2   0.772474     349.992244   6401.341211                2.336785          99.676255            2       True          9
2           LightGBM_BAG_L4   0.772474     605.438185  16222.400024                2.180843         125.503560            4       True         19
3       WeightedEnsemble_L5   0.772474     608.623039  16439.443824                3.184853         217.043800            5       True         21
4       WeightedEnsemble_L4   0.772466     606.422412  16388.884723                3.165069         291.988259            4       True         17
5           LightGBM_BAG_L3   0.772429     488.800207  11806.385116                1.867947          84.587803            3 

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.772486,412.501170,9597.861271,3.178522,282.661976,3,True,12
1,LightGBM_BAG_L2,0.772474,349.992244,6401.341211,2.336785,99.676255,2,True,9
2,LightGBM_BAG_L4,0.772474,605.438185,16222.400024,2.180843,125.503560,4,True,19
3,WeightedEnsemble_L5,0.772474,608.623039,16439.443824,3.184853,217.043800,5,True,21
4,WeightedEnsemble_L4,0.772466,606.422412,16388.884723,3.165069,291.988259,4,True,17
5,LightGBM_BAG_L3,0.772429,488.800207,11806.385116,1.867947,84.587803,3,True,14
6,LightGBMXT_BAG_L4,0.772173,605.049316,16191.174558,1.791974,94.278094,4,True,18
7,LightGBMXT_BAG_L3,0.771546,489.102540,11823.016953,2.170280,101.219639,3,True,13
8,RandomForestGini_BAG_L3,0.769963,557.965920,13924.915574,71.033660,2203.118260,3,True,15
9,RandomForestEntr_BAG_L3,0.769123,528.185455,13707.970762,41.253196,1986.173448,3,True,16


In [10]:
preds_autogluon = model_ag.predict_proba(TabularDataset(test))

In [11]:
## create submission
submission = pd.DataFrame({
    'id': preds_autogluon.index,
    'claim': preds_autogluon[1]
})

submission.head()

,id,claim
id,,
957919,957919,0.530387
957920,957920,0.356863
957921,957921,0.538292
957922,957922,0.359884
957923,957923,0.361850


In [12]:
## save submission
PATH_AUTOGLUON_SUBMISSION = 'submission_autogluon_' + str(time_limit) + '.csv'
PATH_AUTOGLUON_SUBMISSION

'submission_autogluon_20000.csv'

In [13]:
submission.to_csv(PATH_AUTOGLUON_SUBMISSION, index=False)

In [14]:
## clear memory
shutil.rmtree('AutogluonModels')

gc.collect()

196